In [1]:
import pandas as pd
import numpy as np
import time
import warnings
from multiprocessing import Pool, cpu_count

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
ruta = "BasedeDatosMoviesEDA.csv"

for enc in ["utf-8-sig", "cp1252", "latin-1", "ISO-8859-1"]:
    try:
        df = pd.read_csv(
            ruta,
            engine="python",  
            sep=None,         
            on_bad_lines="skip",
            encoding=enc
        )
        print("Cargó con encoding:", enc)
        break
    except UnicodeDecodeError as e:
        print("Falló con", enc, "->", e)

# Mostrar información del DataFrame
print(df.shape)
display(df.head())
print(df.info())

Falló con utf-8-sig -> 'utf-8' codec can't decode byte 0xbf in position 4903: invalid start byte
Falló con cp1252 -> 'charmap' codec can't decode byte 0x81 in position 3051: character maps to <undefined>
Cargó con encoding: latin-1
(1048522, 6)


,id,title,vote_average,release_date,budget,popularity
0,27205,Inception,8.364,15/7/2010,160000000.0,83.952
1,157336,Interstellar,8.417,5/11/2014,165000000.0,140.241
2,155,The Dark Knight,8.512,16/7/2008,185000000.0,130.643
3,19995,Avatar,7.573,15/12/2009,237000000.0,79.932
4,24428,The Avengers,7.710,25/4/2012,220000000.0,98.082


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048522 entries, 0 to 1048521
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   id            1048522 non-null  object 
 1   title         1048509 non-null  object 
 2   vote_average  1048266 non-null  float64
 3   release_date  822352 non-null   object 
 4   budget        1048266 non-null  float64
 5   popularity    1048266 non-null  object 
dtypes: float64(2), object(4)
memory usage: 48.0+ MB
None


In [3]:
import numpy as np

# Se simula una variación del 5% a 15%
df["variacion"] = (df["budget"] * (1 + np.random.uniform(0.05, 0.15, len(df)))).round(2)

# Se simula un impuesto o descuento
df["impuesto"] = np.random.choice([0.05, 0.10, 0.15], size=len(df)).round(2)

# Se calcula un valor final 
df["valor_final"] = (df["variacion"] * (1 - df["impuesto"])).round(2)

# Ordenar por valor_final de mayor a menor
df = df.sort_values(by="valor_final", ascending=False)

# Imprimir el DataFrame resultante
print(df[["title", "budget", "variacion", "impuesto", "valor_final"]].head(10))


                                                 title       budget  \
814773                                  In Whose Name?  999999999.0   
372161   ryan gosling der sutter logan pauls tissemand  900000000.0   
419198                                            Enea  888000000.0   
673082                      ETERNIA: TWILIGHT OF STARS  873000000.0   
573175                             Shadow the Hedgehog  800000000.0   
526341                         Adventures in Bora Bora  800000000.0   
553423         Playmobil The Movie 2: Maximus' Revenge  600000000.0   
1005196                                       Avatar 4  500000000.0   
653959           The Arcana Imperium: Sovereign's Fall  490000000.0   
669701                           Red Dead Redemption 2  500000000.0   

            variacion  impuesto   valor_final  
814773   1.127990e+09      0.10  1.015191e+09  
372161   1.007528e+09      0.05  9.571514e+08  
419198   9.604907e+08      0.10  8.644416e+08  
673082   9.970140e+08     

In [4]:
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i][1] <= right[j][1]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result

In [5]:
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2][1]
    left = [x for x in arr if x[1] < pivot]
    middle = [x for x in arr if x[1] == pivot]
    right = [x for x in arr if x[1] > pivot]
    return quick_sort(left) + middle + quick_sort(right)


In [6]:
def ordenar_dataframe(df, algoritmo):
    """Ordena un DataFrame según 'valor_final' usando el algoritmo dado."""
    arr = list(zip(df.index, df["valor_final"].values))
    inicio = time.perf_counter()
    if algoritmo == "mergesort":
        ordenado = merge_sort(arr)
    elif algoritmo == "quicksort":
        ordenado = quick_sort(arr)
    else:
        raise ValueError("Algoritmo no reconocido. Usa 'mergesort' o 'quicksort'.")
    fin = time.perf_counter()

    indices_ordenados = [i for i, _ in ordenado]
    df_ordenado = df.loc[indices_ordenados].reset_index(drop=True)
    return df_ordenado, fin - inicio


In [7]:
print("\n--- ORDENAMIENTO SIN MULTIPROCESAMIENTO ---")

df_merge, tiempo_merge = ordenar_dataframe(df, "mergesort")
print(f" MergeSort completado en {tiempo_merge:.3f} segundos")

df_quick, tiempo_quick = ordenar_dataframe(df, "quicksort")
print(f" QuickSort completado en {tiempo_quick:.3f} segundos")

print("------------------------------------------------")

if tiempo_merge < tiempo_quick:
    print(f" El algoritmo más rápido fue: MergeSort ({tiempo_merge:.3f}s)")
else:
    print(f" El algoritmo más rápido fue: QuickSort ({tiempo_quick:.3f}s)")


--- ORDENAMIENTO SIN MULTIPROCESAMIENTO ---
 MergeSort completado en 2.872 segundos
 QuickSort completado en 0.377 segundos
------------------------------------------------
 El algoritmo más rápido fue: QuickSort (0.377s)


In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def worker_mergesort(chunk):
    arr = list(zip(chunk.index, chunk["valor_final"].values))
    ordenado = merge_sort(arr)
    indices_ordenados = [i for i, _ in ordenado]
    return chunk.loc[indices_ordenados]

def worker_quicksort(chunk):
    arr = list(zip(chunk.index, chunk["valor_final"].values))
    ordenado = quick_sort(arr)
    indices_ordenados = [i for i, _ in ordenado]
    return chunk.loc[indices_ordenados]

def ejecutar_multiproceso(df, algoritmo):
    num_procesos = min(4, cpu_count())
    # repartir todas las filas, sin perder sobrantes
    chunks = [c for c in np.array_split(df, num_procesos) if len(c) > 0]

    print(f"\n Iniciando multiprocesamiento ({algoritmo}) con {len(chunks)} hilos...")
    t0 = time.perf_counter()

    if algoritmo == "mergesort":
        worker_func = worker_mergesort
    elif algoritmo == "quicksort":
        worker_func = worker_quicksort
    else:
        raise ValueError("Algoritmo inválido.")

    with ThreadPoolExecutor(max_workers=len(chunks)) as executor:
        futures = [executor.submit(worker_func, chunk) for chunk in chunks]
        partes = [future.result() for future in as_completed(futures)]

    # concatenar todas las partes; reset_index si quieres índice secuencial
    df_final = pd.concat(partes).reset_index(drop=True)
    t1 = time.perf_counter()

    tiempo_total = t1 - t0
    print(f" {algoritmo.capitalize()} paralelo completado en {tiempo_total:.3f} segundos.")
    return df_final, tiempo_total

In [9]:
print("\n--- ORDENAMIENTO CON MULTIPROCESAMIENTO ---")

df_merge_multi, tiempo_merge_multi = ejecutar_multiproceso(df, "mergesort")
print(f" MergeSort paralelo completado en {tiempo_merge_multi:.3f} segundos")

df_quick_multi, tiempo_quick_multi = ejecutar_multiproceso(df, "quicksort")
print(f" QuickSort paralelo completado en {tiempo_quick_multi:.3f} segundos")

print("------------------------------------------------")
print("\n--- COMPARACIÓN DE RENDIMIENTO ---")
print(f"MergeSort (sin multiproceso): {tiempo_merge:.3f}s")
print(f"MergeSort (con multiproceso): {tiempo_merge_multi:.3f}s")
print(f"Mejora: {((tiempo_merge - tiempo_merge_multi) / tiempo_merge * 100):.2f}%")

print(f"\nQuickSort (sin multiproceso): {tiempo_quick:.3f}s")
print(f"QuickSort (con multiproceso): {tiempo_quick_multi:.3f}s")
print(f"Mejora: {((tiempo_quick - tiempo_quick_multi) / tiempo_quick * 100):.2f}%")


--- ORDENAMIENTO CON MULTIPROCESAMIENTO ---

 Iniciando multiprocesamiento (mergesort) con 4 hilos...
 Mergesort paralelo completado en 3.505 segundos.
 MergeSort paralelo completado en 3.505 segundos

 Iniciando multiprocesamiento (quicksort) con 4 hilos...
 Quicksort paralelo completado en 1.136 segundos.
 QuickSort paralelo completado en 1.136 segundos
------------------------------------------------

--- COMPARACIÓN DE RENDIMIENTO ---
MergeSort (sin multiproceso): 2.872s
MergeSort (con multiproceso): 3.505s
Mejora: -22.03%

QuickSort (sin multiproceso): 0.377s
QuickSort (con multiproceso): 1.136s
Mejora: -201.53%
